In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('DSA_features.csv')

In [3]:
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,8.280854,34.1980,-2.9038,28.080803,5.299132,1.350075,-1.491537,11.2240,-11.65100,14.670334,...,0.200829,-0.040701,0.297666,0.708480,-0.117430,4.135451e-02,0.203358,-0.310022,basketBall,p8
9116,9.591118,51.6970,-3.4129,35.722025,5.976791,2.981144,0.086304,6.9951,-11.76400,5.329897,...,0.148745,-0.266377,0.224716,0.554670,-0.250950,3.355704e-02,0.183186,-0.736410,basketBall,p8
9117,9.599113,27.9300,-1.0765,48.850886,6.989341,0.449237,-0.728367,3.7801,-8.36910,5.683022,...,0.310748,-0.009505,-0.237786,0.088854,-0.477260,2.026107e-02,0.142341,0.668438,basketBall,p8
9118,9.692482,72.7820,-2.6734,59.378336,7.705734,4.491114,-0.582724,6.1216,-8.85710,4.162963,...,0.156493,0.050624,0.533023,0.677800,0.055941,1.356379e-02,0.116464,-1.482489,basketBall,p8


In [4]:
df['activity'].value_counts()

activity
sitting                    480
walkingTreadmillIncline    480
jumping                    480
rowing                     480
cyclingVertical            480
cyclingHorizontal          480
crossTrainer               480
stepper                    480
runningTreadmill           480
walkingTreadmillFlat       480
standing                   480
walkingLot                 480
movingInElevator           480
standingInElevatorStill    480
decendingStairs            480
ascendingStairs            480
lyingRigh                  480
lyingBack                  480
basketBall                 480
Name: count, dtype: int64

In [5]:
df.isnull().values.any()

False

In [13]:
test_data = df[df['activity'].isin(['lyingRigh', 'lyingBack', 'jumping'])].copy()
test_data['abnormal'] = test_data['activity'].apply(lambda x: 0 if x in ['lyingRigh', 'lyingBack'] else 1)

In [14]:
from sklearn.preprocessing import LabelEncoder

le_people = LabelEncoder()
test_data['people'] = le_people.fit_transform(test_data['people'])

In [15]:
train_data = df[df['activity'].isin(['lyingRigh', 'lyingBack'])].copy()
train_data['abnormal'] = 0

In [27]:
X_train = train_data.drop(['people', 'activity', 'abnormal'], axis=1).values
y_train = train_data['abnormal'].values

In [17]:
X_test = test_data.drop(['people', 'activity', 'abnormal'], axis=1)
y_test = test_data['abnormal']

In [18]:
input_dim = X_train.shape[1]
encoding_dim = 32 

In [19]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(input_layer, decoder)

In [20]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [21]:
autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.7501 - val_loss: 5.3346
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - loss: 5.1365 - val_loss: 5.0319
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8492 - val_loss: 4.9947
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9398 - val_loss: 4.9722
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9928 - val_loss: 4.9557
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - loss: 4.7766 - val_loss: 4.9463
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9623 - val_loss: 4.9422
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8503 - val_loss: 4.9386
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8438 - val_loss: 4.9362
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - loss: 4.8196 - val_loss: 4.9330
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8317 - val_loss: 4.9283
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - l

In [22]:
X_test_pred = autoencoder.predict(X_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step


In [23]:
reconstruction_error = ((X_test - X_test_pred) ** 2).mean(axis=1)

In [24]:
threshold = reconstruction_error.mean() + 3 * reconstruction_error.std()

y_pred = (reconstruction_error > threshold).astype(int)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.69
